In [87]:
%pip install -q openpyxl
%pip install -q pyarrow
import hashlib

^C
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pip/__main__.py", line 24, in <module>
    sys.exit(_main())
             ~~~~~^^
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pip/_internal/cli/main.py", line 78, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/importlib/__init__.py", line 88, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importl

In [89]:
# Initial Functions Setup
def normalize_name(name):
    """Normalize tennis player names for matching"""
    if pd.isna(name):
        return ""
    name = str(name).replace('.', '').lower()
    parts = name.split()
    if len(parts) < 2:
        return name.replace(' ', '_')
    if len(parts[-1]) == 1:  # Last part is single letter (first initial)
        last_name = parts[-2]
        first_initial = parts[-1]
    else:  # Handle "First Lastname" format
        last_name = parts[-1]
        first_initial = parts[0][0] if parts[0] else ''
    return f"{last_name}_{first_initial}"

def normalize_jeff_name(name):
    """Normalize Jeff's player names for matching"""
    if pd.isna(name):
        return ""
    name = str(name).lower()
    parts = name.split()
    if len(parts) < 2:
        return name.replace(' ', '_')
    last_name = parts[-1]
    first_initial = parts[0][0] if parts[0] else ''
    return f"{last_name}_{first_initial}"

def normalize_tournament_name(name):
    """Normalize tournament names"""
    if pd.isna(name):
        return ""
    name = str(name).lower()
    name = name.replace('masters cup', 'masters')
    name = name.replace('atp finals', 'masters')
    name = name.replace('wta finals', 'masters')
    return name.strip()

# ------------------------------------------------------------------
# Composite-ID helper
# ------------------------------------------------------------------
def build_composite_id(date_val, tournament_canonical,
                       winner_canonical, loser_canonical):
    """
    Stable 16-character hash to identify the same match
    across different data sources.
    """
    key = f"{date_val}_{tournament_canonical}_{winner_canonical}_{loser_canonical}"
    return hashlib.sha1(key.encode()).hexdigest()[:16]

def load_excel_data(file_path):
    """Load data from Excel file"""
    try:
        df = pd.read_excel(file_path)
        if 'Date' not in df.columns:
            print(f"Warning: No Date column in {file_path}")
            return pd.DataFrame()
        print(f"Loaded {len(df)} matches from {file_path}")
        return df
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return pd.DataFrame()

def get_tournament_tier_weight(tournament_name: str) -> float:
    """Classify tournament tier and return weight"""
    if pd.isna(tournament_name):
        return 0.5
    tournament_lower = tournament_name.lower()
    if any(slam in tournament_lower for slam in ['roland garros', 'wimbledon', 'australian open', 'us open']):
        return 1.0
    masters_events = ['indian wells', 'miami', 'monte carlo', 'madrid', 'rome', 'canada', 'cincinnati', 'shanghai', 'paris masters']
    if any(masters in tournament_lower for masters in masters_events):
        return 0.9
    atp500_events = ['stuttgart', 'barcelona', 'hamburg', 'halle', 'queens', 'washington', 'dubai', 'rotterdam']
    if any(event in tournament_lower for event in atp500_events):
        return 0.7
    if any(lower_tier in tournament_lower for lower_tier in ['itf', 'challenger', 'juniors']):
        return 0.2
    return 0.5

def calculate_recency_weight(match_date, reference_date='2025-07-01'):
    """Calculate exponential decay weight based on match recency"""
    try:
        if isinstance(match_date, str):
            match_dt = datetime.strptime(match_date, '%Y%m%d')
        else:
            match_dt = match_date
        ref_dt = datetime.strptime(reference_date, '%Y-%m-%d')
        days_ago = (ref_dt - match_dt).days
        return np.exp(-0.0005 * days_ago)
    except:
        return 0.5

def load_jeff_comprehensive_data():
    """Load all of Jeff's comprehensive tennis data"""
    base_path = os.path.expanduser("~/Desktop/data/Jeff 6.14.25")
    data = {'men': {}, 'women': {}}
    files = {
        'matches': 'charting-{}-matches.csv',
        'points_2020s': 'charting-{}-points-2020s.csv',
        'overview': 'charting-{}-stats-Overview.csv',
        'serve_basics': 'charting-{}-stats-ServeBasics.csv',
        'return_outcomes': 'charting-{}-stats-ReturnOutcomes.csv',
        'return_depth': 'charting-{}-stats-ReturnDepth.csv',
        'key_points_serve': 'charting-{}-stats-KeyPointsServe.csv',
        'key_points_return': 'charting-{}-stats-KeyPointsReturn.csv',
        'net_points': 'charting-{}-stats-NetPoints.csv',
        'rally': 'charting-{}-stats-Rally.csv',
        'serve_direction': 'charting-{}-stats-ServeDirection.csv',
        'serve_influence': 'charting-{}-stats-ServeInfluence.csv',
        'shot_direction': 'charting-{}-stats-ShotDirection.csv',
        'shot_dir_outcomes': 'charting-{}-stats-ShotDirOutcomes.csv',
        'shot_types': 'charting-{}-stats-ShotTypes.csv',
        'snv': 'charting-{}-stats-SnV.csv',
        'sv_break_split': 'charting-{}-stats-SvBreakSplit.csv',
        'sv_break_total': 'charting-{}-stats-SvBreakTotal.csv'
    }

    for gender in ['men', 'women']:
        gender_path = os.path.join(base_path, gender)
        if os.path.exists(gender_path):
            for key, filename_template in files.items():
                filename = filename_template.format('m' if gender == 'men' else 'w')
                file_path = os.path.join(gender_path, filename)
                if os.path.exists(file_path):
                    df = pd.read_csv(file_path, low_memory=False)
                    if 'player' in df.columns:
                        df['Player_canonical'] = df['player'].apply(normalize_jeff_name)
                    data[gender][key] = df
                    print(f"Loaded {gender}/{filename}: {len(df)} records")
    return data

def load_all_tennis_data():
    """Load tennis data from all years"""
    base_path = os.path.expanduser("~/Desktop/data")
    all_data = []

    for gender_name, gender_code in [("tennisdata_men", "M"), ("tennisdata_women", "W")]:
        gender_path = os.path.join(base_path, gender_name)
        if os.path.exists(gender_path):
            for year in range(2020, 2026):
                file_path = os.path.join(gender_path, f"{year}_{gender_code.lower()}.xlsx")
                if os.path.exists(file_path):
                    df = load_excel_data(file_path)
                    if not df.empty and 'Date' in df.columns:
                        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
                        df['gender'] = gender_code
                        df['year'] = df['Date'].dt.year
                        all_data.append(df)

    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        return pd.DataFrame()

ERROR! Session/line number was not unique in database. History logging moved to new session 46


In [91]:
# ============================================================================
# TENNIS DATA PIPELINE
# ============================================================================
import numpy as np
import pandas as pd
import os
import requests
import json
import pickle
import shutil
import time
from datetime import datetime, date, timedelta
from pathlib import Path
import re
from unidecode import unidecode
import time

# ============================================================================
# API CONFIGURATION AND CORE FUNCTIONS
# ============================================================================

# API-Tennis configuration
API_KEY = "adfc70491c47895e5fffdc6428bbf36a561989d4bffcfa9ecfba8d91e947b4fb"
BASE = "https://api.api-tennis.com/tennis/"
CACHE_API = Path.home() / ".api_tennis_cache"
CACHE_API.mkdir(exist_ok=True)

def api_call(method: str, **params):
    """Unified API call function with proper error handling"""
    try:
        response = requests.get(BASE, params={"method": method, "APIkey": API_KEY, **params}, timeout=10)
        response.raise_for_status()
        data = response.json()

        error_code = str(data.get("error", "0"))
        if error_code != "0":
            return []

        return data.get("result", [])
    except Exception as e:
        print(f"API call failed for {method}: {e}")
        return []

# ============================================================================
# NAME NORMALIZATION FUNCTIONS
# ============================================================================

def normalize_name(name):
    """Normalize tennis player names for matching"""
    if pd.isna(name):
        return ""
    name = str(name).replace('.', '').lower()
    parts = name.split()
    if len(parts) < 2:
        return name.replace(' ', '_')
    if len(parts[-1]) == 1:  # Last part is single letter (first initial)
        last_name = parts[-2]
        first_initial = parts[-1]
    else:  # Handle "First Lastname" format
        last_name = parts[-1]
        first_initial = parts[0][0] if parts[0] else ''
    return f"{last_name}_{first_initial}"

def normalize_jeff_name(name):
    """Normalize Jeff's player names for matching"""
    if pd.isna(name):
        return ""
    name = str(name).lower()
    parts = name.split()
    if len(parts) < 2:
        return name.replace(' ', '_')
    last_name = parts[-1]
    first_initial = parts[0][0] if parts[0] else ''
    return f"{last_name}_{first_initial}"

def normalize_tournament_name(name):
    """Normalize tournament names"""
    if pd.isna(name):
        return ""
    name = str(name).lower()
    name = name.replace('masters cup', 'masters')
    name = name.replace('atp finals', 'masters')
    name = name.replace('wta finals', 'masters')
    return name.strip()

# ============================================================================
# DATA LOADING FUNCTIONS
# ============================================================================

def load_excel_data(file_path):
    """Load data from Excel file"""
    try:
        df = pd.read_excel(file_path)
        if 'Date' not in df.columns:
            print(f"Warning: No Date column in {file_path}")
            return pd.DataFrame()
        print(f"Loaded {len(df)} matches from {file_path}")
        return df
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return pd.DataFrame()

def load_jeff_comprehensive_data():
    """Load all of Jeff's comprehensive tennis data"""
    base_path = os.path.expanduser("~/Desktop/data/Jeff 6.14.25")
    data = {'men': {}, 'women': {}}
    files = {
        'matches': 'charting-{}-matches.csv',
        'points_2020s': 'charting-{}-points-2020s.csv',
        'overview': 'charting-{}-stats-Overview.csv',
        'serve_basics': 'charting-{}-stats-ServeBasics.csv',
        'return_outcomes': 'charting-{}-stats-ReturnOutcomes.csv',
        'return_depth': 'charting-{}-stats-ReturnDepth.csv',
        'key_points_serve': 'charting-{}-stats-KeyPointsServe.csv',
        'key_points_return': 'charting-{}-stats-KeyPointsReturn.csv',
        'net_points': 'charting-{}-stats-NetPoints.csv',
        'rally': 'charting-{}-stats-Rally.csv',
        'serve_direction': 'charting-{}-stats-ServeDirection.csv',
        'serve_influence': 'charting-{}-stats-ServeInfluence.csv',
        'shot_direction': 'charting-{}-stats-ShotDirection.csv',
        'shot_dir_outcomes': 'charting-{}-stats-ShotDirOutcomes.csv',
        'shot_types': 'charting-{}-stats-ShotTypes.csv',
        'snv': 'charting-{}-stats-SnV.csv',
        'sv_break_split': 'charting-{}-stats-SvBreakSplit.csv',
        'sv_break_total': 'charting-{}-stats-SvBreakTotal.csv'
    }

    for gender in ['men', 'women']:
        gender_path = os.path.join(base_path, gender)
        if os.path.exists(gender_path):
            for key, filename_template in files.items():
                filename = filename_template.format('m' if gender == 'men' else 'w')
                file_path = os.path.join(gender_path, filename)
                if os.path.exists(file_path):
                    df = pd.read_csv(file_path, low_memory=False)
                    if 'player' in df.columns:
                        df['Player_canonical'] = df['player'].apply(normalize_jeff_name)
                    data[gender][key] = df
                    print(f"Loaded {gender}/{filename}: {len(df)} records")
    return data

def load_all_tennis_data():
    """Load tennis data from all years"""
    base_path = os.path.expanduser("~/Desktop/data")
    all_data = []

    for gender_name, gender_code in [("tennisdata_men", "M"), ("tennisdata_women", "W")]:
        gender_path = os.path.join(base_path, gender_name)
        if os.path.exists(gender_path):
            for year in range(2020, 2026):
                file_path = os.path.join(gender_path, f"{year}_{gender_code.lower()}.xlsx")
                if os.path.exists(file_path):
                    df = load_excel_data(file_path)
                    if not df.empty and 'Date' in df.columns:
                        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
                        df['gender'] = gender_code
                        df['year'] = df['Date'].dt.year
                        all_data.append(df)

    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        return pd.DataFrame()

# ============================================================================
# FEATURE EXTRACTION AND DEFAULTS
# ============================================================================

def get_fallback_defaults(gender_key):
    """Fallback defaults when no Jeff data available"""
    base_defaults = {
        'serve_pts': 80, 'aces': 6, 'double_faults': 3, 'first_serve_pct': 0.62,
        'first_serve_won': 35, 'second_serve_won': 16, 'break_points_saved': 4,
        'return_pts_won': 30, 'winners_total': 28, 'winners_fh': 16, 'winners_bh': 12,
        'unforced_errors': 28, 'unforced_fh': 16, 'unforced_bh': 12,
        'serve_wide_pct': 0.3, 'serve_t_pct': 0.4, 'serve_body_pct': 0.3,
        'return_deep_pct': 0.4, 'return_shallow_pct': 0.3, 'return_very_deep_pct': 0.2,
        'key_points_serve_won_pct': 0.6, 'key_points_aces_pct': 0.05, 'key_points_first_in_pct': 0.55,
        'key_points_return_won_pct': 0.35, 'key_points_return_winners': 0.02,
        'net_points_won_pct': 0.65, 'net_winners_pct': 0.3, 'passed_at_net_pct': 0.3,
        'rally_server_winners_pct': 0.15, 'rally_server_unforced_pct': 0.2,
        'rally_returner_winners_pct': 0.1, 'rally_returner_unforced_pct': 0.25,
        'shot_crosscourt_pct': 0.5, 'shot_down_line_pct': 0.25, 'shot_inside_out_pct': 0.15,
        'serve_volley_frequency': 0.02, 'serve_volley_success_pct': 0.6,
        'return_error_net_pct': 0.1, 'return_error_wide_pct': 0.05,
        'aggression_index': 0.5, 'consistency_index': 0.5, 'pressure_performance': 0.5, 'net_game_strength': 0.5
    }

    if gender_key == 'women':
        base_defaults.update({
            'serve_pts': 75, 'aces': 4, 'first_serve_pct': 0.60,
            'first_serve_won': 32, 'second_serve_won': 15,
            'serve_volley_frequency': 0.01, 'net_points_won_pct': 0.60
        })

    return base_defaults

def calculate_comprehensive_weighted_defaults(jeff_data):
    """Calculate weighted defaults from all Jeff datasets"""
    print("Calculating comprehensive weighted defaults from Jeff's data...")
    defaults = {'men': {}, 'women': {}}

    for gender in ['men', 'women']:
        if gender not in jeff_data:
            continue

        print(f"\nProcessing {gender}'s comprehensive data...")
        matches_df = jeff_data[gender].get('matches')
        if matches_df is None:
            print(f"No matches data for {gender}")
            continue

        gender_defaults = {}

        # Overview stats - basic serving/returning
        if 'overview' in jeff_data[gender]:
            overview_df = jeff_data[gender]['overview']
            match_totals = overview_df[overview_df['set'] == 'Total'].copy()

            # Simple defaults from median values
            if len(match_totals) > 0:
                gender_defaults.update({
                    'serve_pts': float(match_totals['serve_pts'].median()) if 'serve_pts' in match_totals.columns else 77.0,
                    'aces': float(match_totals['aces'].median()) if 'aces' in match_totals.columns else 5.0,
                    'double_faults': float(match_totals['dfs'].median()) if 'dfs' in match_totals.columns else 3.0,
                    'first_serve_pct': 0.62,  # reasonable default
                    'first_serve_won': float(match_totals['first_won'].median()) if 'first_won' in match_totals.columns else 35.0,
                    'second_serve_won': float(match_totals['second_won'].median()) if 'second_won' in match_totals.columns else 16.0,
                    'break_points_saved': float(match_totals['bp_saved'].median()) if 'bp_saved' in match_totals.columns else 4.0,
                    'return_pts_won': float(match_totals['return_pts_won'].median()) if 'return_pts_won' in match_totals.columns else 30.0,
                    'winners_total': float(match_totals['winners'].median()) if 'winners' in match_totals.columns else 28.0,
                    'winners_fh': float(match_totals['winners_fh'].median()) if 'winners_fh' in match_totals.columns else 16.0,
                    'winners_bh': float(match_totals['winners_bh'].median()) if 'winners_bh' in match_totals.columns else 12.0,
                    'unforced_errors': float(match_totals['unforced'].median()) if 'unforced' in match_totals.columns else 28.0,
                    'unforced_fh': float(match_totals['unforced_fh'].median()) if 'unforced_fh' in match_totals.columns else 16.0,
                    'unforced_bh': float(match_totals['unforced_bh'].median()) if 'unforced_bh' in match_totals.columns else 12.0,
                })

        # Add fallback defaults for missing features
        fallback = get_fallback_defaults(gender)
        for key, value in fallback.items():
            if key not in gender_defaults:
                gender_defaults[key] = float(value)

        defaults[gender] = gender_defaults
        print(f"Calculated defaults for {gender}: {len(gender_defaults)} features")

    return defaults

def extract_comprehensive_jeff_features(player_canonical, gender, jeff_data, weighted_defaults=None):
    """Extract features from all Jeff datasets with Player_canonical checks"""
    gender_key = 'men' if gender == 'M' else 'women'

    if gender_key not in jeff_data:
        return get_fallback_defaults(gender_key)

    if weighted_defaults and gender_key in weighted_defaults:
        features = weighted_defaults[gender_key].copy()
    else:
        features = get_fallback_defaults(gender_key)

    # Overview stats
    if 'overview' in jeff_data[gender_key]:
        overview_df = jeff_data[gender_key]['overview']
        if 'Player_canonical' in overview_df.columns:
            player_overview = overview_df[
                (overview_df['Player_canonical'] == player_canonical) &
                (overview_df['set'] == 'Total')
            ]

            if len(player_overview) > 0:
                latest = player_overview.iloc[-1]
                serve_pts = latest.get('serve_pts', 80)
                if serve_pts > 0:
                    features.update({
                        'serve_pts': float(serve_pts),
                        'aces': float(latest.get('aces', 0)),
                        'double_faults': float(latest.get('dfs', 0)),
                        'first_serve_pct': float(latest.get('first_in', 0)) / float(serve_pts) if serve_pts > 0 else 0.62,
                        'first_serve_won': float(latest.get('first_won', 0)),
                        'second_serve_won': float(latest.get('second_won', 0)),
                        'break_points_saved': float(latest.get('bp_saved', 0)),
                        'return_pts_won': float(latest.get('return_pts_won', 0)),
                        'winners_total': float(latest.get('winners', 0)),
                        'winners_fh': float(latest.get('winners_fh', 0)),
                        'winners_bh': float(latest.get('winners_bh', 0)),
                        'unforced_errors': float(latest.get('unforced', 0)),
                        'unforced_fh': float(latest.get('unforced_fh', 0)),
                        'unforced_bh': float(latest.get('unforced_bh', 0))
                    })

    return features

# ============================================================================
# API-TENNIS HELPER FUNCTIONS
# ============================================================================

def get_fixtures_for_date(target_date):
    """Get all fixtures for a specific date - includes embedded statistics"""
    try:
        fixtures = api_call("get_fixtures",
                           date_start=target_date.isoformat(),
                           date_stop=target_date.isoformat(),
                           timezone="UTC")

        finished_fixtures = [ev for ev in fixtures if ev.get("event_status") == "Finished"]
        return finished_fixtures
    except Exception as e:
        print(f"Error getting fixtures for {target_date}: {e}")
        return []


def extract_embedded_statistics(fixture):
    """Extract statistics from fixture data (no separate API call needed)"""
    stats = {}

    # Extract from scores data
    scores = fixture.get("scores", [])
    if scores:
        try:
            p1_sets = 0
            p2_sets = 0
            for s in scores:
                score_first = safe_int_convert(s.get("score_first", 0), 0)
                score_second = safe_int_convert(s.get("score_second", 0), 0)
                if score_first > score_second:
                    p1_sets += 1
                elif score_second > score_first:
                    p2_sets += 1

            stats["sets_won_p1"] = p1_sets
            stats["sets_won_p2"] = p2_sets
            stats["total_sets"] = len(scores)
        except Exception as e:
            # Don't fail the entire match for score parsing errors
            pass

    return stats


# ----------------------------------------------------------------------------
# API-TENNIS: Statistics parsing helper
# ----------------------------------------------------------------------------
def parse_match_statistics(fixture):
    """
    Parse the raw ``fixture["statistics"]`` list into per–player dictionaries.

    Parameters
    ----------
    fixture : dict
        A single fixture dictionary returned by the API.  We expect a key
        ``"statistics"`` that is a list of dictionaries, each containing at
        least ``"player_key"``, ``"type"``, and ``"value"``.

    Returns
    -------
    dict
        ``{player_key: {stat_name: value, ...}, ...}``
        where *stat_name* is the lower‑snake‑case version of the original
        ``type`` string and *value* is converted to ``float`` when possible.
    """
    stats_pair = {}

    for entry in fixture.get("statistics", []):
        p_key = safe_int_convert(entry.get("player_key"))
        if p_key is None:
            continue

        # Normalise statistic name: "First Serves In" -> "first_serves_in"
        stat_name = str(entry.get("type", "")).strip().lower().replace(" ", "_")
        if not stat_name:
            continue

        raw_val = entry.get("value")
        try:
            val = float(raw_val)
        except (TypeError, ValueError):
            val = raw_val  # leave as‑is if non‑numeric

        if p_key not in stats_pair:
            stats_pair[p_key] = {}
        stats_pair[p_key][stat_name] = val

    return stats_pair

def get_match_odds(match_key, date_check=None):
    """Get odds with proper error handling - only for dates >= 2025-06-23"""
    if date_check and date_check < date(2025, 6, 23):
        return (None, None)

    try:
        # Convert match_key to int for API call, but keep as string for lookup
        match_key_int = safe_int_convert(match_key)
        if match_key_int is None:
            return (None, None)

        odds_data = api_call("get_odds", match_key=match_key_int)
        if not odds_data or str(match_key_int) not in odds_data:
            return (None, None)

        match_odds = odds_data[str(match_key_int)]
        home_away = match_odds.get("Home/Away", {})

        # Get average odds across bookmakers
        home_odds = home_away.get("Home", {})
        away_odds = home_away.get("Away", {})

        if home_odds and away_odds:
            # Take first bookmaker's odds or average if multiple
            home_val = next(iter(home_odds.values())) if home_odds else None
            away_val = next(iter(away_odds.values())) if away_odds else None

            return (float(home_val) if home_val else None,
                   float(away_val) if away_val else None)

        return (None, None)
    except Exception as e:
        print(f"Error getting odds for match {match_key}: {e}")
        return (None, None)

def safe_int_convert(value, default=None):
    """Safely convert string/float to int, handling decimals and None values"""
    if value is None or value == "":
        return default
    try:
        # Convert to float first to handle decimal strings like "7.7", then to int
        return int(float(str(value)))
    except (ValueError, TypeError):
        return default

def get_player_rankings(day, league="ATP"):
    """Get standings with proper caching and error handling"""
    tag = f"{league}_{day.isocalendar()[0]}_{day.isocalendar()[1]:02d}.pkl"
    cache_file = CACHE_API / tag

    if cache_file.exists():
        try:
            standings = pickle.loads(cache_file.read_bytes())
            if standings:  # Non-empty cache
                rankings = {}
                for r in standings:
                    player_key = safe_int_convert(r.get("player_key"))
                    place = safe_int_convert(r.get("place"))
                    if player_key is not None and place is not None:
                        rankings[player_key] = place
                return rankings
        except Exception as e:
            print(f"Cache read error for {tag}: {e}")

    # Correct parameter for API
    standings = api_call("get_standings", event_type=league.upper())

    try:
        cache_file.write_bytes(pickle.dumps(standings, 4))
    except Exception as e:
        print(f"Cache write error for {tag}: {e}")

    # Process standings with safe conversion
    rankings = {}
    for r in standings:
        player_key = safe_int_convert(r.get("player_key"))
        place = safe_int_convert(r.get("place"))
        if player_key is not None and place is not None:
            rankings[player_key] = place

    return rankings

def get_h2h_data(p1_key, p2_key):
    """Get head-to-head data with caching"""
    cache_file = CACHE_API / f"h2h_{p1_key}_{p2_key}.pkl"

    if cache_file.exists():
        try:
            return pickle.loads(cache_file.read_bytes())
        except Exception:
            pass

    try:
        h2h_data = api_call("get_H2H", first_player_key=p1_key, second_player_key=p2_key)

        # Handle empty response or error
        if not h2h_data or not isinstance(h2h_data, list) or len(h2h_data) == 0:
            result = {"h2h_matches": 0, "p1_wins": 0, "p2_wins": 0, "p1_win_pct": 0.5}
        else:
            h2h_matches = h2h_data[0].get("H2H", []) if isinstance(h2h_data[0], dict) else []
            if not h2h_matches:
                result = {"h2h_matches": 0, "p1_wins": 0, "p2_wins": 0, "p1_win_pct": 0.5}
            else:
                p1_wins = sum(1 for m in h2h_matches if m.get("event_winner") == "First Player")
                p2_wins = len(h2h_matches) - p1_wins

                result = {
                    "h2h_matches": len(h2h_matches),
                    "p1_wins": p1_wins,
                    "p2_wins": p2_wins,
                    "p1_win_pct": p1_wins / len(h2h_matches) if h2h_matches else 0.5
                }

        cache_file.write_bytes(pickle.dumps(result, 4))
        return result

    except Exception as e:
        result = {"h2h_matches": 0, "p1_wins": 0, "p2_wins": 0, "p1_win_pct": 0.5}
        cache_file.write_bytes(pickle.dumps(result, 4))
        return result

def get_tournaments_metadata():
    """Get tournament metadata (surface, level, location) - cached statically"""
    cache_file = CACHE_API / "tournaments.pkl"

    if cache_file.exists():
        try:
            return pickle.loads(cache_file.read_bytes())
        except Exception:
            pass

    try:
        tournaments = api_call("get_tournaments")
        # Use safe conversion for tournament_key
        tournament_dict = {}
        for t in tournaments:
            tournament_key = safe_int_convert(t.get("tournament_key"))
            if tournament_key is not None:
                tournament_dict[str(tournament_key)] = t  # Keep as string for lookup
        cache_file.write_bytes(pickle.dumps(tournament_dict, 4))
        return tournament_dict
    except Exception as e:
        print(f"Error getting tournaments: {e}")
        return {}

def get_event_types():
    """Get event types metadata - cached statically"""
    cache_file = CACHE_API / "event_types.pkl"

    if cache_file.exists():
        try:
            return pickle.loads(cache_file.read_bytes())
        except Exception:
            pass

    try:
        events = api_call("get_events")
        # Use safe conversion for event_type_key
        event_dict = {}
        for e in events:
            event_type_key = safe_int_convert(e.get("event_type_key"))
            if event_type_key is not None:
                event_dict[str(event_type_key)] = e  # Keep as string for lookup
        cache_file.write_bytes(pickle.dumps(event_dict, 4))
        return event_dict
    except Exception as e:
        print(f"Error getting event types: {e}")
        return {}

def get_player_profile(player_key):
    """Get player profile with career stats"""
    cache_file = CACHE_API / f"player_{player_key}.pkl"

    if cache_file.exists():
        try:
            return pickle.loads(cache_file.read_bytes())
        except Exception:
            pass

    try:
        player_data = api_call("get_players", player_key=player_key)
        result = player_data[0] if player_data else {}
        cache_file.write_bytes(pickle.dumps(result, 4))
        return result
    except Exception as e:
        print(f"Error getting player {player_key}: {e}")
        return {}

# ============================================================================
# MAIN DATA GENERATION FUNCTION
# ============================================================================

def generate_comprehensive_historical_data(fast=True, n_sample=500):
    """Generate comprehensive historical data with API integration"""
    print("=== STARTING DATA GENERATION ===")

    # Step 1: Load Jeff's data
    print("Step 1: Loading Jeff's comprehensive data...")
    try:
        jeff_data = load_jeff_comprehensive_data()
        if not jeff_data or ('men' not in jeff_data and 'women' not in jeff_data):
            print("ERROR: Jeff data loading failed")
            return pd.DataFrame(), {}, {}

        print(f"✓ Jeff data loaded successfully")
        print(f"  - Men's datasets: {len(jeff_data.get('men', {}))}")
        print(f"  - Women's datasets: {len(jeff_data.get('women', {}))}")

    except Exception as e:
        print(f"ERROR loading Jeff data: {e}")
        return pd.DataFrame(), {}, {}

    # Step 2: Calculate weighted defaults
    print("Step 2: Calculating weighted defaults...")
    try:
        weighted_defaults = calculate_comprehensive_weighted_defaults(jeff_data)
        if not weighted_defaults:
            print("ERROR: Weighted defaults calculation failed")
            return pd.DataFrame(), jeff_data, {}

        print(f"✓ Weighted defaults calculated")
        print(f"  - Men's features: {len(weighted_defaults.get('men', {}))}")
        print(f"  - Women's features: {len(weighted_defaults.get('women', {}))}")

    except Exception as e:
        print(f"ERROR calculating weighted defaults: {e}")
        return pd.DataFrame(), jeff_data, {}

    # Step 3: Load tennis match data
    print("Step 3: Loading tennis match data...")
    try:
        tennis_data = load_all_tennis_data()
        if tennis_data.empty:
            print("ERROR: No tennis data loaded")
            return pd.DataFrame(), jeff_data, weighted_defaults

        print(f"✓ Tennis data loaded: {len(tennis_data)} matches")

        # Fast mode for testing
        if fast:
            total_rows = len(tennis_data)
            take = min(n_sample, total_rows)
            tennis_data = tennis_data.sample(take, random_state=1).reset_index(drop=True)
            print(f"[FAST MODE] Using sample of {take}/{total_rows} rows")

    except Exception as e:
        print(f"ERROR loading tennis data: {e}")
        return pd.DataFrame(), jeff_data, weighted_defaults

    # Step 4: Process tennis data
    print("Step 4: Processing tennis data...")
    try:
        # Normalize player names
        tennis_data['winner_canonical'] = tennis_data['Winner'].apply(normalize_name)
        tennis_data['loser_canonical'] = tennis_data['Loser'].apply(normalize_name)
        tennis_data['tournament_canonical'] = tennis_data['Tournament'].apply(normalize_tournament_name)
        # --- fix: ensure Date columns exist before building composite_id ---
        tennis_data['Date'] = pd.to_datetime(tennis_data['Date'], errors='coerce')
        tennis_data['date'] = tennis_data['Date'].dt.date

        # ------------------------------------------------------------------
        # Composite match identifier shared across data sources
        # ------------------------------------------------------------------
        tennis_data['composite_id'] = tennis_data.apply(
            lambda r: build_composite_id(
                r['date'],
                r['tournament_canonical'],
                r['winner_canonical'],
                r['loser_canonical']
            ),
            axis=1
        )

        # Add odds data
        tennis_data['tennis_data_odds1'] = pd.to_numeric(tennis_data.get('PSW', 0), errors='coerce')
        tennis_data['tennis_data_odds2'] = pd.to_numeric(tennis_data.get('PSL', 0), errors='coerce')

        # Add ranking difference
        if 'WRank' in tennis_data.columns and 'LRank' in tennis_data.columns:
            tennis_data['rank_difference'] = abs(pd.to_numeric(tennis_data['WRank'], errors='coerce') -
                                                 pd.to_numeric(tennis_data['LRank'], errors='coerce'))

        print(f"✓ Tennis data processed")

    except Exception as e:
        print(f"ERROR processing tennis data: {e}")
        return pd.DataFrame(), jeff_data, weighted_defaults

    # Step 5: Add Jeff feature columns
    print("Step 5: Adding Jeff feature columns...")
    try:
        all_jeff_features = [
            'serve_pts', 'aces', 'double_faults', 'first_serve_pct', 'first_serve_won',
            'second_serve_won', 'break_points_saved', 'return_pts_won',
            'winners_total', 'winners_fh', 'winners_bh', 'unforced_errors', 'unforced_fh', 'unforced_bh',
            'serve_wide_pct', 'serve_t_pct', 'serve_body_pct',
            'return_deep_pct', 'return_shallow_pct', 'return_very_deep_pct',
            'key_points_serve_won_pct', 'key_points_aces_pct', 'key_points_first_in_pct',
            'key_points_return_won_pct', 'key_points_return_winners',
            'net_points_won_pct', 'net_winners_pct', 'passed_at_net_pct',
            'rally_server_winners_pct', 'rally_server_unforced_pct',
            'rally_returner_winners_pct', 'rally_returner_unforced_pct',
            'shot_crosscourt_pct', 'shot_down_line_pct', 'shot_inside_out_pct',
            'serve_volley_frequency', 'serve_volley_success_pct',
            'return_error_net_pct', 'return_error_wide_pct',
            'aggression_index', 'consistency_index', 'pressure_performance', 'net_game_strength'
        ]

        # Initialize feature columns
        for feature in all_jeff_features:
            tennis_data[f'winner_{feature}'] = pd.Series(dtype='float64')
            tennis_data[f'loser_{feature}'] = pd.Series(dtype='float64')

        print(f"✓ Added {len(all_jeff_features) * 2} feature columns")

    except Exception as e:
        print(f"ERROR adding feature columns: {e}")
        return pd.DataFrame(), jeff_data, weighted_defaults

    # Step 6: Extract Jeff features
    print("Step 6: Extracting Jeff features...")
    try:
        total_matches = len(tennis_data)
        matches_with_jeff_features = 0

        for idx, row in tennis_data.iterrows():
            if idx % 100 == 0:
                print(f"  Processing match {idx}/{total_matches}")

            try:
                gender = row['gender']

                # Only extract Jeff features for matches before cutoff
                if row['date'] <= date(2025, 6, 10):
                    winner_features = extract_comprehensive_jeff_features(
                        row['winner_canonical'], gender, jeff_data, weighted_defaults
                    )
                    loser_features = extract_comprehensive_jeff_features(
                        row['loser_canonical'], gender, jeff_data, weighted_defaults
                    )

                    # Assign features
                    for feature_name, feature_value in winner_features.items():
                        col_name = f'winner_{feature_name}'
                        if col_name in tennis_data.columns:
                            tennis_data.at[idx, col_name] = feature_value

                    for feature_name, feature_value in loser_features.items():
                        col_name = f'loser_{feature_name}'
                        if col_name in tennis_data.columns:
                            tennis_data.at[idx, col_name] = feature_value

                    if winner_features and loser_features:
                        matches_with_jeff_features += 1

            except Exception as e:
                if idx < 5:  # Only print first few errors
                    print(f"  Warning: Error processing match {idx}: {e}")
                continue

        print(f"✓ Jeff features extracted for {matches_with_jeff_features}/{total_matches} matches")

    except Exception as e:
        print(f"ERROR extracting Jeff features: {e}")
        return pd.DataFrame(), jeff_data, weighted_defaults

    print(f"=== DATA GENERATION COMPLETE ===")
    print(f"Final data shape: {tennis_data.shape}")

    return tennis_data, jeff_data, weighted_defaults

# ============================================================================
# CACHE FUNCTIONS
# ============================================================================

CACHE_DIR = os.path.expanduser("~/Desktop/data/cache")
HD_PATH = os.path.join(CACHE_DIR, "historical_data.parquet")
JEFF_PATH = os.path.join(CACHE_DIR, "jeff_data.pkl")
DEF_PATH = os.path.join(CACHE_DIR, "weighted_defaults.pkl")

def save_to_cache(historical_data, jeff_data, weighted_defaults):
    """Save data to cache"""
    print("\n=== SAVING TO CACHE ===")
    # --- sanitize numeric columns that sometimes contain stray text ---
    numeric_cols = ["MaxW", "MaxL", "AvgW", "AvgL", "PSW", "PSL"]
    for col in numeric_cols:
        if col in historical_data.columns:
            historical_data[col] = pd.to_numeric(historical_data[col], errors="coerce")
    os.makedirs(CACHE_DIR, exist_ok=True)

    try:
        # Save historical data
        historical_data.to_parquet(HD_PATH, index=False)
        print("✓ Historical data saved")

        # Save Jeff data
        with open(JEFF_PATH, "wb") as f:
            pickle.dump(jeff_data, f, protocol=pickle.HIGHEST_PROTOCOL)
        print("✓ Jeff data saved")

        # Save weighted defaults
        with open(DEF_PATH, "wb") as f:
            pickle.dump(weighted_defaults, f, protocol=pickle.HIGHEST_PROTOCOL)
        print("✓ Weighted defaults saved")

        return True
    except Exception as e:
        print(f"ERROR saving cache: {e}")
        return False

def load_from_cache():
    """Load data from cache if available"""
    if (os.path.exists(HD_PATH) and
        os.path.exists(JEFF_PATH) and
        os.path.exists(DEF_PATH)):

        print("Loading from cache...")
        historical_data = pd.read_parquet(HD_PATH)

        with open(JEFF_PATH, "rb") as f:
            jeff_data = pickle.load(f)

        with open(DEF_PATH, "rb") as f:
            weighted_defaults = pickle.load(f)

        return historical_data, jeff_data, weighted_defaults
    else:
        return None, None, None

# ============================================================================
# API INTEGRATION FUNCTIONS
# ============================================================================

from datetime import date, timedelta

def integrate_api_tennis_data(historical_data, days_back: int = 3):
    """Integrate API-Tennis data for recent matches.

    Parameters
    ----------
    historical_data : DataFrame
        Existing combined dataset.
    days_back : int, default 3
        How many days before *today* to start pulling finished matches.
    """
    start_date = date.today() - timedelta(days=days_back)
    print("Step 7: Integrating comprehensive API-Tennis data...")

    # Load static metadata once
    print("  Loading tournament and event type metadata...")
    tournaments_meta = get_tournaments_metadata()
    event_types_meta = get_event_types()
    print(f"  Loaded {len(tournaments_meta)} tournaments, {len(event_types_meta)} event types")

    # Ensure event_key column exists
    if "event_key" not in historical_data.columns:
        historical_data["event_key"] = pd.NA

    # Get existing API keys to avoid duplicates
    existing_keys = set()
    if len(historical_data) > 0:
        cutoff_data = historical_data[historical_data["date"] >= start_date]
        if len(cutoff_data) > 0:
            # Safely convert existing event_keys to integers
            for key in cutoff_data["event_key"].dropna():
                converted_key = safe_int_convert(key)
                if converted_key is not None:
                    existing_keys.add(converted_key)
    # ------------------------------------------------------------------
    # Ensure composite_id column and back-fill where missing
    # ------------------------------------------------------------------
    if "composite_id" not in historical_data.columns:
        historical_data["composite_id"] = pd.NA

    if historical_data['composite_id'].isna().any():
        historical_data.loc[historical_data['composite_id'].isna(), 'composite_id'] = (
            historical_data[historical_data['composite_id'].isna()]
            .apply(lambda r: build_composite_id(
                r['date'],
                r.get('tournament_canonical', normalize_tournament_name(r['Tournament'])),
                r.get('winner_canonical',  normalize_name(r['Winner'])),
                r.get('loser_canonical',   normalize_name(r['Loser']))
            ), axis=1)
        )

    print(f"Found {len(existing_keys)} existing API matches")

    api_matches = []
    date_range = list(pd.date_range(start_date, date.today()))

    for d in date_range:
        day = d.date()
        try:
            fixtures = get_fixtures_for_date(day)
            if fixtures:
                print(f"  {day}: {len(fixtures)} finished matches")

            # Build a per‑day dataframe with rich stats for quick lookup
            fixtures_df = flatten_fixtures(fixtures)
            fixtures_df = fixtures_df.set_index("event_key")

            for fixture in fixtures:
                try:
                    event_key = safe_int_convert(fixture.get("event_key"))
                    if event_key is None:
                        continue

                    if event_key in existing_keys:
                        continue

                    # Basic match info
                    p1_name = fixture["event_first_player"]
                    p2_name = fixture["event_second_player"]
                    winner = p1_name if fixture.get("event_winner", "").startswith("First") else p2_name
                    loser = p2_name if winner == p1_name else p1_name

                    # Tournament metadata
                    tournament_key = fixture.get("tournament_key")
                    tournament_meta = tournaments_meta.get(tournament_key, {})

                    # Event type metadata
                    event_type_key = tournament_meta.get("event_type_key")
                    event_type_meta = event_types_meta.get(event_type_key, {})

                    match_record = {
                        "Date": pd.to_datetime(fixture["event_date"]),
                        "date": pd.to_datetime(fixture["event_date"]).date(),
                        "event_key": event_key,
                        "Tournament": fixture.get("tournament_name", "Unknown"),
                        "round": fixture.get("tournament_round", ""),
                        "Surface": fixture.get("court_surface", "Hard"),
                        "Winner": winner,
                        "Loser": loser,
                        "source_rank": 1,  # API source
                        "gender": "W" if "wta" in fixture.get("event_type_type", "").lower() else "M",
                        "composite_id": build_composite_id(
                            match_record["date"],
                            match_record["tournament_canonical"],
                            match_record["winner_canonical"],
                            match_record["loser_canonical"]
                        ),
                    }

                    # Extract embedded statistics from fixture
                    embedded_stats = extract_embedded_statistics(fixture)
                    match_record.update(embedded_stats)

                    # Merge the pre‑flattened wide statistics row for this match
                    if event_key in fixtures_df.index:
                        rich = fixtures_df.loc[event_key].to_dict()
                        for col, val in rich.items():
                            if col not in match_record:      # keep existing core fields
                                match_record[col] = val

                    # Get odds (only for dates >= 2025-06-23)
                    odds1, odds2 = get_match_odds(event_key, day)

                    # Add normalized names
                    match_record["winner_canonical"] = normalize_name(winner)
                    match_record["loser_canonical"] = normalize_name(loser)
                    match_record["tournament_canonical"] = normalize_tournament_name(match_record["Tournament"])

                    # Extract embedded statistics from fixture
                    embedded_stats = extract_embedded_statistics(fixture)
                    match_record.update(embedded_stats)

                    # Add raw fixture data for later analysis
                    match_record["api_scores"] = json.dumps(fixture.get("scores", []))
                    match_record["api_pointbypoint_available"] = len(fixture.get("pointbypoint", [])) > 0

                    # Get odds (only for dates >= 2025-06-23)
                    odds1, odds2 = get_match_odds(event_key, day)
                    match_record["api_odds_home"] = odds1
                    match_record["api_odds_away"] = odds2

                    # Get rankings
                    league = "WTA" if match_record["gender"] == "W" else "ATP"
                    rankings = get_player_rankings(day, league)
                    p1_key = int(fixture.get("first_player_key", 0))
                    p2_key = int(fixture.get("second_player_key", 0))

                    if winner == p1_name:
                        match_record["WRank"] = rankings.get(p1_key, pd.NA)
                        match_record["LRank"] = rankings.get(p2_key, pd.NA)
                        match_record["winner_player_key"] = p1_key
                        match_record["loser_player_key"] = p2_key
                    else:
                        match_record["WRank"] = rankings.get(p2_key, pd.NA)
                        match_record["LRank"] = rankings.get(p1_key, pd.NA)
                        match_record["winner_player_key"] = p2_key
                        match_record["loser_player_key"] = p1_key

                    # Get H2H data
                    h2h_data = get_h2h_data(p1_key, p2_key)
                    match_record.update({f"h2h_{k}": v for k, v in h2h_data.items()})

                    # Add tournament metadata
                    match_record["tournament_key"] = tournament_key
                    match_record["event_type_key"] = event_type_key
                    match_record["event_type_type"] = fixture.get("event_type_type", "")
                    match_record["tournament_season"] = fixture.get("tournament_season", "")

                    api_matches.append(match_record)
                    existing_keys.add(event_key)

                    # Rate limiting
                    time.sleep(0.1)

                except Exception as e:
                    print(f"    Error processing match {fixture.get('event_key', 'unknown')}: {e}")
                    continue

        except Exception as e:
            print(f"  Error processing day {day}: {e}")
            continue

    print(f"Collected {len(api_matches)} new API matches with full metadata")

    # Merge with historical data
    if api_matches:
        try:
            api_df = pd.DataFrame(api_matches)

            # Align columns
            for col in historical_data.columns:
                if col not in api_df.columns:
                    api_df[col] = pd.NA

            for col in api_df.columns:
                if col not in historical_data.columns:
                    historical_data[col] = pd.NA

            # Ensure column order matches
            api_df = api_df.reindex(columns=historical_data.columns, fill_value=pd.NA)

            # Concatenate
            combined_data = pd.concat([historical_data, api_df], ignore_index=True)

            # Add source_rank if missing
            if "source_rank" not in combined_data.columns:
                combined_data["source_rank"] = 2  # Default to tennis-data
            combined_data["source_rank"] = combined_data["source_rank"].fillna(2)

            # Deduplicate (keep API data over tennis-data when available)
            dedup_keys = ["event_key", "composite_id"]
            final_data = (
                combined_data
                .sort_values("source_rank")  # API=1, tennis-data=2
                .drop_duplicates(subset=dedup_keys, keep="first")
                .reset_index(drop=True)
            )

            print(f"✓ Successfully integrated {len(api_df)} API matches")
            print(f"Final dataset: {len(final_data)} matches")

            # Show what we got
            if len(api_df) > 0:
                odds_count = api_df["api_odds_home"].notna().sum()
                h2h_count = api_df["h2h_matches"].notna().sum()
                rankings_count = api_df["WRank"].notna().sum()
                print(f"  - Matches with odds: {odds_count}")
                print(f"  - Matches with H2H data: {h2h_count}")
                print(f"  - Matches with rankings: {rankings_count}")

            return final_data

        except Exception as e:
            print(f"Error merging API data: {e}")
            return historical_data
    else:
        print("No new API data to merge")
        return historical_data

# ============================================================================
# MAIN EXECUTION
# ============================================================================

# ------------------------------------------------------------------
# Cache logic — flip REFRESH_CACHE to force regeneration
# ------------------------------------------------------------------
REFRESH_CACHE = True          # set False to reuse cached data

if REFRESH_CACHE:
    print("Refreshing cache …")
    historical_data, jeff_data, weighted_defaults = generate_comprehensive_historical_data(
        fast=False                         # use full dataset so API rows can merge
    )
    save_to_cache(historical_data, jeff_data, weighted_defaults)
else:
    historical_data, jeff_data, weighted_defaults = load_from_cache()
    if historical_data is None:
        print("Cache miss – generating data …")
        historical_data, jeff_data, weighted_defaults = generate_comprehensive_historical_data(
            fast=False
        )
        save_to_cache(historical_data, jeff_data, weighted_defaults)
    else:
        print("✓ Data loaded from cache")

    # pull finished API‑Tennis matches from 10 June 2025 onward
    days_back = (date.today() - date(2025, 6, 10)).days
    historical_data = integrate_api_tennis_data(historical_data, days_back=days_back)
    save_to_cache(historical_data, jeff_data, weighted_defaults)

Refreshing cache …
=== STARTING DATA GENERATION ===
Step 1: Loading Jeff's comprehensive data...
Loaded men/charting-m-matches.csv: 6639 records
Loaded men/charting-m-points-2020s.csv: 461897 records
Loaded men/charting-m-stats-Overview.csv: 49928 records
Loaded men/charting-m-stats-ServeBasics.csv: 39779 records
Loaded men/charting-m-stats-ReturnOutcomes.csv: 271988 records
Loaded men/charting-m-stats-ReturnDepth.csv: 236950 records
Loaded men/charting-m-stats-KeyPointsServe.csv: 53048 records
Loaded men/charting-m-stats-KeyPointsReturn.csv: 53048 records
Loaded men/charting-m-stats-NetPoints.csv: 52101 records
Loaded men/charting-m-stats-Rally.csv: 84877 records
Loaded men/charting-m-stats-ServeDirection.csv: 39779 records
Loaded men/charting-m-stats-ServeInfluence.csv: 26527 records
Loaded men/charting-m-stats-ShotDirection.csv: 52639 records
Loaded men/charting-m-stats-ShotDirOutcomes.csv: 153266 records
Loaded men/charting-m-stats-ShotTypes.csv: 308946 records
Loaded men/charting-

/var/folders/tn/l2t1vp_14yz0__jghc_2zdb40000gn/T/ipykernel_66988/3741653389.py:697: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tennis_data[f'loser_{feature}'] = pd.Series(dtype='float64')
/var/folders/tn/l2t1vp_14yz0__jghc_2zdb40000gn/T/ipykernel_66988/3741653389.py:696: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tennis_data[f'winner_{feature}'] = pd.Series(dtype='float64')
/var/folders/tn/l2t1vp_14yz0__jghc_2zdb40000gn/T/ipykernel_66988/3741653389.py:697: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

  Processing match 100/25451
  Processing match 200/25451
  Processing match 300/25451
  Processing match 400/25451
  Processing match 500/25451
  Processing match 600/25451
  Processing match 700/25451
  Processing match 800/25451
  Processing match 900/25451
  Processing match 1000/25451
  Processing match 1100/25451
  Processing match 1200/25451
  Processing match 1300/25451
  Processing match 1400/25451
  Processing match 1500/25451
  Processing match 1600/25451
  Processing match 1700/25451
  Processing match 1800/25451
  Processing match 1900/25451
  Processing match 2000/25451
  Processing match 2100/25451
  Processing match 2200/25451
  Processing match 2300/25451
  Processing match 2400/25451
  Processing match 2500/25451
  Processing match 2600/25451
  Processing match 2700/25451
  Processing match 2800/25451
  Processing match 2900/25451
  Processing match 3000/25451
  Processing match 3100/25451
  Processing match 3200/25451
  Processing match 3300/25451
  Processing match 

############# APIIIIII ##############

In [92]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict
import hashlib

# --------------------------------------------------------------------------
# helpers
# --------------------------------------------------------------------------
def canonical_name(name: str) -> str:
    return normalize_name(name)

def winner_from_fixture(fx):
    return (fx["event_first_player"]
            if fx["event_winner"].startswith("First") else
            fx["event_second_player"])

def loser_from_fixture(fx):
    return (fx["event_second_player"]
            if winner_from_fixture(fx) == fx["event_first_player"] else
            fx["event_first_player"])

def parse_score_block(score_list):
    """turn the API 'scores' array into set-by-set ints"""
    sets = []
    for s in score_list:
        try:
            f, l = int(float(s["score_first"])), int(float(s["score_second"]))
        except Exception:
            f, l = np.nan, np.nan
        sets.append((f, l))
    return sets                     # [(7,6),(6,2), …]

# --------------------------------------------------------------------------
# main flattener
# --------------------------------------------------------------------------
def flatten_fixtures(fixtures):

    records = []
    for fx in fixtures:
        # ------------------------------------------------------------------
        # 1.  core match row
        # ------------------------------------------------------------------
        rec = {
            "event_key"   : int(fx["event_key"]),
            "date"        : pd.to_datetime(fx["event_date"]),
            "tournament"  : fx.get("tournament_name"),
            "round"       : fx.get("tournament_round", ""),
            "surface"     : fx.get("court_surface", "Hard"),
            "event_type"  : fx.get("event_type_type"),
            "season"      : fx.get("tournament_season"),
            "winner"      : winner_from_fixture(fx),
            "loser"       : loser_from_fixture(fx),
            "winner_id"   : int(fx["first_player_key"]
                                 if fx["event_winner"].startswith("First")
                                 else fx["second_player_key"]),
            "loser_id"    : int(fx["second_player_key"]
                                 if fx["event_winner"].startswith("First")
                                 else fx["first_player_key"]),
            "winner_canonical": canonical_name(winner_from_fixture(fx)),
            "loser_canonical" : canonical_name(loser_from_fixture(fx)),
            # raw json blobs if you still want them later
            "api_scores_raw"      : json.dumps(fx.get("scores", [])),
            "api_pointbypoint"    : len(fx.get("pointbypoint", [])) > 0,
        }
        # compute composite_id now that rec exists
        rec["composite_id"] = build_composite_id(
            rec["date"].date(),
            normalize_tournament_name(rec["tournament"]),
            rec["winner_canonical"],
            rec["loser_canonical"]
        )

        # ------------------------------------------------------------------
        # 2.  score-level features
        # ------------------------------------------------------------------
        set_scores = parse_score_block(fx.get("scores", []))
        rec["sets_played"] = len(set_scores)
        rec["sets_won_winner"] = sum(f > l for f, l in set_scores)
        rec["games_winner"] = sum(f for f, _ in set_scores)
        rec["games_loser"]  = sum(l for _, l in set_scores)
        rec["tb_played"]    = any(max(f,l) >= 7 for f, l in set_scores)

        # ------------------------------------------------------------------
        # 3.  STATISTICS  →  wide columns
        # ------------------------------------------------------------------
        stat_df = pd.DataFrame(fx.get("statistics", []))
        if not stat_df.empty:
            # build "stat_col" like aces_match, first_serve_pct_set1, …
            stat_df["stat_col"] = (
                stat_df["stat_name"]
                  .str.lower()
                  .str.replace("%","pct")
                  .str.replace(" ", "_")
                + "_" + stat_df["stat_period"]
            )

            # two wide frames, one for each player key
            for pid, tag in [(fx["first_player_key"], "p1"),
                             (fx["second_player_key"], "p2")]:
                sub = (stat_df[stat_df["player_key"] == pid]
                       .drop_duplicates(subset=["stat_col"], keep="last"))

                wide = (sub
                        .pivot_table(index="player_key",
                                     columns="stat_col",
                                     values="stat_value",
                                     aggfunc="first"))
                if not wide.empty:
                    for c, v in wide.iloc[0].items():
                        rec[f"{tag}_{c}"] = v

        records.append(rec)

    wide_df = pd.DataFrame(records)
    # make data types nice
    num_cols = [c for c in wide_df.columns if c.startswith(("p1_","p2_"))]
    wide_df[num_cols] = (wide_df[num_cols]
                         .apply(pd.to_numeric, errors="ignore"))

    return wide_df

# --------------------------------------------------------------------------
# example usage
# --------------------------------------------------------------------------
from datetime import date
fixtures = get_fixtures_for_date(date(2025, 6, 17))  # or any list you built
deep_df  = flatten_fixtures(fixtures)

print(deep_df.shape)
deep_df.head()

(457, 175)


/var/folders/tn/l2t1vp_14yz0__jghc_2zdb40000gn/T/ipykernel_66988/2021403865.py:119: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  .apply(pd.to_numeric, errors="ignore"))


,event_key,date,tournament,round,surface,event_type,season,winner,loser,winner_id,...,p2_net_points_won_set2,p2_net_points_won_set3,p2_unforced_errors_match,p2_unforced_errors_set1,p2_unforced_errors_set2,p2_unforced_errors_set3,p2_winners_match,p2_winners_set1,p2_winners_set2,p2_winners_set3
0,12043380,2025-06-17,Nottingham Challenger Men,Nottingham Challenger Men - 1/16-finals,Hard,Challenger Men Singles,2025,L. Harris,V. Royer,2834,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12043387,2025-06-17,Nottingham Challenger Men,Nottingham Challenger Men - 1/16-finals,Hard,Challenger Men Singles,2025,E. Nava,J. Pinnington Jones,567,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12043388,2025-06-17,Nottingham Challenger Men,Nottingham Challenger Men - 1/16-finals,Hard,Challenger Men Singles,2025,T. Schoolkate,A. Blockx,1183,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12043455,2025-06-17,WTA Berlin,WTA Berlin - 1/16-finals,Hard,Wta Singles,2025,A. Anisimova,B. Andreescu,2074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12043458,2025-06-17,WTA Berlin,WTA Berlin - 1/16-finals,Hard,Wta Singles,2025,L. Samsonova,N. Osaka,68376,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
# ------------------------------------------------------------------
# 1.  harmonise API frame and build robust match key
# ------------------------------------------------------------------
deep_df = deep_df.copy()          # the (457 × …) frame just built

# rename to match tennis‑data column names
deep_df.rename(
    columns={
        "date": "Date",
        "tournament": "Tournament",
        "winner": "Winner",
        "loser": "Loser",
    },
    inplace=True,
)

# ── build canonical/composite identifiers on BOTH tables ────────────
def _canon_name(s: str) -> str:
    return normalize_name(s)

def _canon_tourn(s: str) -> str:
    return normalize_tournament_name(s)

def _build_cid(row) -> str:
    return build_composite_id(
        pd.to_datetime(row["Date"]).date(),
        _canon_tourn(row["Tournament"]),
        _canon_name(row["Winner"]),
        _canon_name(row["Loser"]),
    )

for _df in (historical_data, deep_df):
    _df["Date"] = pd.to_datetime(_df["Date"], errors="coerce")
    _df["composite_id"] = _df.apply(_build_cid, axis=1)

# ── keep only useful API columns ─────────────────────────────────────
core_cols = [
    "event_key", "composite_id", "Date", "Tournament",
    "Winner", "Loser", "sets_played", "sets_won_winner",
    "games_winner", "games_loser", "tb_played",
]
stat_cols = [c for c in deep_df.columns if c.startswith(("p1_", "p2_"))]
api_subset = deep_df[core_cols + stat_cols].copy()

# ------------------------------------------------------------------
# 2.  append API stats and deduplicate
# ------------------------------------------------------------------
# ensure rank column
if "source_rank" not in historical_data.columns:
    historical_data["source_rank"] = 2       # 2 = tennis‑data default

api_subset["source_rank"] = 1                # 1 = API rows

# combine
combined = pd.concat([historical_data, api_subset], ignore_index=True)

# keep API over tennis‑data when duplicate identifiers collide
dedup_keys = ["event_key", "composite_id"]
combined = (
    combined.sort_values("source_rank")      # API first
            .drop_duplicates(subset=dedup_keys, keep="first")
            .reset_index(drop=True)
)

historical_data = combined      # commit back for later cells

# quick sanity
print("combined shape :", combined.shape)
print("API rows now    :", (combined['source_rank'] == 1).sum())

/var/folders/tn/l2t1vp_14yz0__jghc_2zdb40000gn/T/ipykernel_66988/628092459.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  historical_data["source_rank"] = 2       # 2 = tennis‑data default


combined shape : (25908, 295)
API rows now    : 457


In [94]:
# how many api rows?
print(len(api_subset))

# any merge hits?
merged_flag = historical_data.merge(
    api_subset[["event_key"] + key_cols],
    on=key_cols,
    how="left",
    indicator=True
)["_merge"].value_counts()
print(merged_flag)

457
_merge
left_only     25451
both            457
right_only        0
Name: count, dtype: int64


SIMULATION

In [95]:
# LAYER 1 ##
def extract_data_samples():
    # Jeff Sackmann data samples
    jeff_samples = {
        'matches': jeff_data['men']['matches'].head(3),
        'serve_basics': jeff_data['men']['serve_basics'].head(3),
        'overview': jeff_data['men']['overview'].head(3)
    }

    # Tennis-data samples
    tennis_samples = historical_data[
        ['Winner', 'Loser', 'WRank', 'LRank', 'PSW', 'PSL', 'Surface']
    ].head(3)

    return jeff_samples, tennis_samples

# Hold/break computation method verification
hold_break_computation = {
    'current_method': 'Jeff aggregated stats from overview dataset',
    'available_columns': ['serve_pts', 'first_in', 'first_won', 'second_won'],
    'computation_level': 'Per-player aggregate from charting data'
}

# Bayesian
def extract_priors_from_current_data(player_canonical, gender, surface):
    priors = {}

    # Layer 1: Elo approximation from rankings
    player_matches = historical_data[
        (historical_data['winner_canonical'] == player_canonical) |
        (historical_data['loser_canonical'] == player_canonical)
    ]

    if len(player_matches) > 0:
        # Ranking-based Elo estimation
        recent_rank = get_recent_rank(player_canonical, player_matches)
        elo_estimate = 2000 - (recent_rank * 5) if recent_rank else 1500

        # Jeff feature extraction
        jeff_features = extract_jeff_features(player_canonical, gender, jeff_data)

        priors = {
            'elo_estimate': elo_estimate,
            'serve_effectiveness': jeff_features.get('serve_pts', 0.6),
            'return_strength': jeff_features.get('return_pts_won', 0.3),
            'surface_factor': calculate_surface_adjustment(player_matches, surface)
        }

    return priors

# Time decay for recent form
def calculate_time_decayed_performance(player_matches, reference_date):
    player_matches['days_ago'] = (reference_date - player_matches['date']).dt.days

    # Exponential decay: recent matches weighted heavier
    weights = np.exp(-0.01 * player_matches['days_ago'])  # 1% daily decay

    weighted_performance = {
        'win_rate': np.average(player_matches['is_winner'], weights=weights),
        'games_won_rate': np.average(player_matches['games_won_pct'], weights=weights)
    }

    return weighted_performance

In [96]:
## TEST ##
import os, pickle, pandas as pd

CACHE_DIR = os.path.expanduser("~/Desktop/data/cache")
os.makedirs(CACHE_DIR, exist_ok=True)
HD_PATH   = os.path.join(CACHE_DIR, "historical_data.parquet")
JEFF_PATH = os.path.join(CACHE_DIR, "jeff_data.pkl")
DEF_PATH  = os.path.join(CACHE_DIR, "weighted_defaults.pkl")

if (os.path.exists(HD_PATH) and
    os.path.exists(JEFF_PATH) and
    os.path.exists(DEF_PATH)):
    print("Loading cached data …")
    historical_data = pd.read_parquet(HD_PATH)
    with open(JEFF_PATH, "rb") as fh:
        jeff_data = pickle.load(fh)
    with open(DEF_PATH, "rb") as fh:
        weighted_defaults = pickle.load(fh)
else:
    print("Cache miss – regenerating (one-time slow run).")
    combined_data, jeff_data, weighted_defaults = generate_comprehensive_historical_all_years()
    historical_data = combined_data
    historical_data.to_parquet(HD_PATH, index=False)
    with open(JEFF_PATH, "wb") as fh:
        pickle.dump(jeff_data, fh, protocol=pickle.HIGHEST_PROTOCOL)
    with open(DEF_PATH, "wb") as fh:
        pickle.dump(weighted_defaults, fh, protocol=pickle.HIGHEST_PROTOCOL)

Loading cached data …


"SIMULATION"

In [97]:
import pandas as pd
import numpy as np
from collections import defaultdict

def normalize_name_canonical(name):
    if pd.isna(name):
        return ""
    name = str(name).strip()
    name = name.replace('.', '').replace("'", '').replace('-', ' ')
    return ' '.join(name.lower().split())

def extract_jeff_features(player_canonical, gender, jeff_data):
    """Extract actual features from Jeff Sackmann data"""
    gender_key = 'men' if gender == 'M' else 'women'

    if gender_key not in jeff_data or player_canonical not in jeff_data[gender_key]:
        return {
            'serve_pts': 60,
            'first_won': 0,
            'second_won': 0,
            'return_pts_won': 20
        }

    player_data = jeff_data[gender_key][player_canonical]

    first_in = player_data.get('1stIn', 0)
    first_won = player_data.get('1stWon', 0)
    second_won = player_data.get('2ndWon', 0)
    double_faults = player_data.get('df', 0)

    total_serve_pts = first_in + double_faults + (first_won - first_in) if first_won >= first_in else first_in + second_won + double_faults

    break_points_saved = player_data.get('bpSaved', 0)
    break_points_faced = player_data.get('bpFaced', 0)
    return_pts_won = break_points_faced - break_points_saved

    return {
        'serve_pts': max(1, total_serve_pts),
        'first_won': first_won,
        'second_won': second_won,
        'return_pts_won': max(0, return_pts_won)
    }

class BayesianTennisModel:
    def __init__(self):
        self.simulation_count = 10000
        self.jeff_data = jeff_data
        self.historical_data = historical_data

    def default_priors(self):
        return {
            'elo_mean': 1500,
            'elo_std': 200,
            'hold_prob': 0.65,
            'break_prob': 0.35,
            'surface': 'Hard',
            'form_factor': 1.0,
            'confidence': 0.1
        }

    def extract_refined_priors(self, player_canonical, gender, surface, reference_date):
        player_matches = self.historical_data[
            (self.historical_data['winner_canonical'] == player_canonical) |
            (self.historical_data['loser_canonical'] == player_canonical)
        ].copy()

        if len(player_matches) == 0:
            return self.default_priors()

        surface_matches = player_matches[player_matches['Surface'] == surface]
        if len(surface_matches) < 5:
            surface_matches = player_matches

        recent_matches = surface_matches.tail(20).copy()
        recent_matches['days_ago'] = (pd.to_datetime(reference_date) - pd.to_datetime(recent_matches['Date'])).dt.days
        weights = np.exp(-0.05 * recent_matches['days_ago'])

        base_elo = self.get_player_weighted_elo(player_canonical, surface, reference_date)
        surface_factor = self.calculate_surface_adaptation(player_canonical, surface)
        elo_prior = base_elo * surface_factor

        jeff_features = extract_jeff_features(player_canonical, gender, self.jeff_data)

        serve_pts = jeff_features['serve_pts']
        serve_won = jeff_features['first_won'] + jeff_features['second_won']
        hold_prob = serve_won / serve_pts if serve_pts > 0 else 0.65

        return_pts = jeff_features['return_pts_won']
        total_return_pts = serve_pts
        break_prob = (1 - return_pts / total_return_pts) if total_return_pts > 0 else 0.35

        return {
            'elo_mean': elo_prior,
            'elo_std': 150,
            'hold_prob': min(0.95, max(0.3, hold_prob)),
            'break_prob': max(0.05, min(0.7, break_prob)),
            'surface': surface,
            'form_factor': self.calculate_form_spike(recent_matches, weights, player_canonical),
            'confidence': max(0.05, min(1.0, len(recent_matches) / 15))
        }

    def calculate_ranking_differential_odds(self, p1_ranking, p2_ranking):
        """Convert ranking differential to implied probability"""
        if p1_ranking == 0 or p2_ranking == 0:
            return 0.5

        ranking_diff = p2_ranking - p1_ranking

        if ranking_diff > 50:
            return 0.85
        elif ranking_diff > 20:
            return 0.75
        elif ranking_diff > 10:
            return 0.65
        elif ranking_diff > 0:
            return 0.55
        elif ranking_diff > -10:
            return 0.45
        elif ranking_diff > -20:
            return 0.35
        elif ranking_diff > -50:
            return 0.25
        else:
            return 0.15

    def calculate_upset_frequency(self, ranking_diff, surface, historical_data):
        """Calculate upset frequency by ranking differential and surface"""
        upset_matches = historical_data[
            ((historical_data['WRank'] - historical_data['LRank']) > ranking_diff) &
            (historical_data['Surface'] == surface)
        ]

        total_matches = historical_data[
            (abs(historical_data['WRank'] - historical_data['LRank']) >= abs(ranking_diff)) &
            (historical_data['Surface'] == surface)
        ]

        if len(total_matches) < 10 and surface != 'fallback':
            return self.calculate_upset_frequency(ranking_diff, 'fallback', historical_data)

        if surface == 'fallback':
            upset_matches = historical_data[
                (historical_data['WRank'] - historical_data['LRank']) > ranking_diff
            ]
            total_matches = historical_data[
                abs(historical_data['WRank'] - historical_data['LRank']) >= abs(ranking_diff)
            ]

        if len(total_matches) == 0:
            return 0.1

        upset_rate = len(upset_matches) / len(total_matches)
        return min(0.45, max(0.05, upset_rate))

    def calculate_surface_performance_ratio(self, player_canonical, surface, opponent_canonical, reference_date):
        """Calculate player's surface-specific performance vs opponent's baseline"""
        player_surface_matches = self.historical_data[
            ((self.historical_data['winner_canonical'] == player_canonical) |
             (self.historical_data['loser_canonical'] == player_canonical)) &
            (self.historical_data['Surface'] == surface) &
            (pd.to_datetime(self.historical_data['Date']) <= pd.to_datetime(reference_date))
        ].tail(20)

        opponent_surface_matches = self.historical_data[
            ((self.historical_data['winner_canonical'] == opponent_canonical) |
             (self.historical_data['loser_canonical'] == opponent_canonical)) &
            (self.historical_data['Surface'] == surface) &
            (pd.to_datetime(self.historical_data['Date']) <= pd.to_datetime(reference_date))
        ].tail(20)

        if len(player_surface_matches) < 3 or len(opponent_surface_matches) < 3:
            return 1.0

        player_wins = len(player_surface_matches[player_surface_matches['winner_canonical'] == player_canonical])
        opponent_wins = len(opponent_surface_matches[opponent_surface_matches['winner_canonical'] == opponent_canonical])

        player_ratio = player_wins / len(player_surface_matches)
        opponent_ratio = opponent_wins / len(opponent_surface_matches)

        return player_ratio / opponent_ratio if opponent_ratio > 0 else 1.0

    def run_simulation(self, p1_priors, p2_priors, iterations):
        return [self.simulate_match(p1_priors, p2_priors)]

    def predict_match_outcome(self, player1_canonical, player2_canonical, surface, gender, date):
        p1_priors = self.extract_refined_priors(player1_canonical, gender, surface, date)
        p2_priors = self.extract_refined_priors(player2_canonical, gender, surface, date)

        base_prob = self.run_simulation(p1_priors, p2_priors, 1000)[0]

        p1_rank = self.get_player_ranking(player1_canonical, date)
        p2_rank = self.get_player_ranking(player2_canonical, date)
        ranking_prob = self.calculate_ranking_differential_odds(p1_rank, p2_rank)

        ranking_diff = p1_rank - p2_rank
        upset_adjustment = self.calculate_upset_frequency(ranking_diff, surface, self.historical_data)

        surface_ratio = self.calculate_surface_performance_ratio(player1_canonical, surface, player2_canonical, date)

        calibrated_prob = (0.6 * base_prob + 0.25 * ranking_prob + 0.15 * surface_ratio) * (1 - upset_adjustment * 0.1)

        return max(0.05, min(0.95, calibrated_prob))

    def get_player_ranking(self, player_canonical, date):
        """Get player ranking at specific date"""
        date_obj = pd.to_datetime(date)

        player_matches = self.historical_data[
            ((self.historical_data['winner_canonical'] == player_canonical) |
             (self.historical_data['loser_canonical'] == player_canonical)) &
            (pd.to_datetime(self.historical_data['Date']) <= date_obj)
        ].sort_values('Date', ascending=False)

        if len(player_matches) == 0:
            return 999

        latest_match = player_matches.iloc[0]

        if latest_match['winner_canonical'] == player_canonical:
            return latest_match.get('WRank', 999)
        else:
            return latest_match.get('LRank', 999)

    def calculate_match_probability(self, player1_canonical, player2_canonical, gender, surface, reference_date, best_of=3):
        player1_priors = self.extract_refined_priors(player1_canonical, gender, surface, reference_date)
        player2_priors = self.extract_refined_priors(player2_canonical, gender, surface, reference_date)

        probability = self.simulate_match(player1_priors, player2_priors, best_of)
        confidence = min(player1_priors['confidence'], player2_priors['confidence'])

        return {
            'player1_win_probability': probability,
            'player2_win_probability': 1 - probability,
            'confidence': confidence,
            'player1_priors': player1_priors,
            'player2_priors': player2_priors
        }

    def calculate_form_spike(self, recent_matches, weights, player_canonical):
        if len(recent_matches) == 0:
            return 1.0

        wins = (recent_matches['winner_canonical'] == player_canonical).astype(int)
        weighted_win_rate = np.average(wins, weights=weights)

        avg_opponent_rank = recent_matches['LRank'].fillna(recent_matches['WRank']).mean()
        player_rank = recent_matches['WRank'].fillna(recent_matches['LRank']).iloc[-1]

        if pd.notna(avg_opponent_rank) and pd.notna(player_rank):
            rank_diff = player_rank - avg_opponent_rank
            expected_win_rate = 1 / (1 + 10**(rank_diff/400))
            form_spike = min(2.0, weighted_win_rate / max(0.1, expected_win_rate))
        else:
            form_spike = 1.0

        return form_spike

    def simulate_match(self, player1_priors, player2_priors, best_of=3):
        wins = 0
        for _ in range(self.simulation_count):
            sets_won = [0, 0]
            while max(sets_won) < (best_of + 1) // 2:
                set_winner = self.simulate_set(player1_priors, player2_priors)
                sets_won[set_winner] += 1
            if sets_won[0] > sets_won[1]:
                wins += 1
        return wins / self.simulation_count

    def simulate_set(self, p1_priors, p2_priors):
        games = [0, 0]
        server = 0
        while True:
            hold_prob = p1_priors['hold_prob'] if server == 0 else p2_priors['hold_prob']
            game_winner = server if np.random.random() < hold_prob else 1 - server
            games[game_winner] += 1
            server = 1 - server
            if games[0] >= 6 and games[0] - games[1] >= 2:
                return 0
            elif games[1] >= 6 and games[1] - games[0] >= 2:
                return 1
            elif games[0] == 6 and games[1] == 6:
                return self.simulate_tiebreak(p1_priors, p2_priors)

    def simulate_tiebreak(self, p1_priors, p2_priors):
        points = [0, 0]
        server = 0
        serve_count = 0
        while True:
            hold_prob = p1_priors['hold_prob'] if server == 0 else p2_priors['hold_prob']
            point_winner = server if np.random.random() < hold_prob else 1 - server
            points[point_winner] += 1
            serve_count += 1
            if serve_count == 1 or serve_count % 2 == 0:
                server = 1 - server
            if points[0] >= 7 and points[0] - points[1] >= 2:
                return 0
            elif points[1] >= 7 and points[1] - points[0] >= 2:
                return 1

    def get_player_weighted_elo(self, player_canonical, surface, reference_date):
        recent_match = self.historical_data[
            ((self.historical_data['winner_canonical'] == player_canonical) |
             (self.historical_data['loser_canonical'] == player_canonical)) &
            (self.historical_data['Surface'] == surface)
        ].tail(1)

        if len(recent_match) > 0 and 'BlendScore' in recent_match.columns:
            blend_score = recent_match['BlendScore'].iloc[0]
            return 1500 + blend_score * 50

        any_surface_match = self.historical_data[
            (self.historical_data['winner_canonical'] == player_canonical) |
            (self.historical_data['loser_canonical'] == player_canonical)
        ].tail(1)

        if len(any_surface_match) > 0 and 'BlendScore' in any_surface_match.columns:
            return 1500 + any_surface_match['BlendScore'].iloc[0] * 200

        return 1500

    def calculate_surface_adaptation(self, player_canonical, target_surface):
        player_matches = self.historical_data[
            (self.historical_data['winner_canonical'] == player_canonical) |
            (self.historical_data['loser_canonical'] == player_canonical)
        ].copy()

        if len(player_matches) < 10:
            return 1.0

        surface_matches = player_matches[player_matches['Surface'] == target_surface]
        if len(surface_matches) < 3:
            return 1.0

        surface_wins = (surface_matches['winner_canonical'] == player_canonical).sum()
        surface_win_rate = surface_wins / len(surface_matches)

        total_wins = (player_matches['winner_canonical'] == player_canonical).sum()
        baseline_win_rate = total_wins / len(player_matches)

        if baseline_win_rate == 0:
            return 1.0

        adaptation_ratio = surface_win_rate / baseline_win_rate
        return max(0.7, min(1.5, adaptation_ratio))

    def evaluate_predictions(self, test_data):
        """Evaluate model accuracy on test dataset"""
        correct = 0
        total = 0

        for _, match in test_data.iterrows():
            prob = self.predict_match_outcome(
                match['winner_canonical'],
                match['loser_canonical'],
                match['Surface'],
                match['gender'],
                match['Date']
            )

            predicted_winner = match['winner_canonical'] if prob > 0.5 else match['loser_canonical']
            actual_winner = match['winner_canonical']

            if predicted_winner == actual_winner:
                correct += 1
            total += 1

        return correct / total if total > 0 else 0

def convert_to_canonical(name):
    return normalize_name_canonical(name)

model = BayesianTennisModel()

In [98]:
## LAYER 2 ##
def apply_contextual_adjustments(self, priors, player_canonical, opponent_canonical, match_context):
    """Layer 2: Contextual Bayesian adjustments for fatigue, injury, motivation"""

    adjusted_priors = priors.copy()

    # Fatigue Index
    fatigue_penalty = self.calculate_fatigue_index(player_canonical, match_context['reference_date'])
    adjusted_priors['hold_prob'] *= (1 - fatigue_penalty * 0.15)  # Max 15% hold penalty
    adjusted_priors['elo_std'] *= (1 + fatigue_penalty * 0.3)    # Increase uncertainty

    # Injury Flag Adjustment
    injury_factor = self.get_injury_factor(player_canonical, match_context['reference_date'])
    adjusted_priors['hold_prob'] *= injury_factor
    adjusted_priors['break_prob'] *= (2 - injury_factor)  # Inverse relationship

    # Form Spike Sustainability
    form_sustainability = self.calculate_form_sustainability(player_canonical, match_context)
    if adjusted_priors['form_factor'] > 1.2:  # Hot streak detection
        sustainability_discount = 1 - ((adjusted_priors['form_factor'] - 1) * (1 - form_sustainability))
        adjusted_priors['hold_prob'] *= sustainability_discount
        adjusted_priors['elo_mean'] *= sustainability_discount

    # Opponent Quality Weighting
    opponent_elo = self.estimate_opponent_elo(opponent_canonical, match_context)
    elo_diff = adjusted_priors['elo_mean'] - opponent_elo
    quality_adjustment = 1 / (1 + np.exp(-elo_diff / 200))  # Sigmoid scaling
    adjusted_priors['break_prob'] *= quality_adjustment

    return adjusted_priors

def calculate_fatigue_index(self, player_canonical, reference_date):
    """Fatigue based on recent match load and recovery time"""
    recent_matches = self.get_recent_matches(player_canonical, reference_date, days=14)

    if len(recent_matches) == 0:
        return 0.0

    # Calculate cumulative fatigue
    fatigue_score = 0
    for _, match in recent_matches.iterrows():
        days_ago = (pd.to_datetime(reference_date) - pd.to_datetime(match['Date'])).days
        match_duration = match.get('minutes', 120)  # Default 2 hours

        # Exponential decay with match duration weighting
        fatigue_contribution = (match_duration / 60) * np.exp(-0.1 * days_ago)
        fatigue_score += fatigue_contribution

    return min(1.0, fatigue_score / 10)  # Normalize to 0-1

def get_injury_factor(self, player_canonical, reference_date):
    """Player-specific injury fragility scoring"""
    # Injury memory bank - replace with actual injury tracking
    injury_prone_players = {
        'nadal_r': 0.85,
        'murray_a': 0.80,
        'thiem_d': 0.75,
        'badosa_p': 0.70
    }

    base_factor = injury_prone_players.get(player_canonical, 0.95)

    # Check for recent retirement/walkover flags
    recent_retirements = self.check_recent_retirements(player_canonical, reference_date)
    if recent_retirements > 0:
        base_factor *= (0.8 ** recent_retirements)

    return max(0.5, base_factor)

def calculate_form_sustainability(self, player_canonical, match_context):
    """Form spike sustainability based on opponent quality and win quality"""
    recent_matches = self.get_recent_matches(player_canonical, match_context['reference_date'], days=21)

    if len(recent_matches) < 3:
        return 0.5

    # Quality-weighted recent performance
    quality_scores = []
    for _, match in recent_matches.iterrows():
        opponent_rank = match['LRank'] if match['winner_canonical'] == player_canonical else match['WRank']
        win_quality = 1 / (1 + opponent_rank / 100) if pd.notna(opponent_rank) else 0.5
        quality_scores.append(win_quality)

    avg_opponent_quality = np.mean(quality_scores)
    consistency = 1 - np.std(quality_scores)

    return min(1.0, avg_opponent_quality * consistency)

def estimate_opponent_elo(self, opponent_canonical, match_context):
    """Quick opponent Elo estimation for quality weighting"""
    opponent_priors = self.extract_refined_priors(
        opponent_canonical,
        match_context['gender'],
        match_context['surface'],
        match_context['reference_date']
    )
    return opponent_priors['elo_mean']

def get_recent_matches(self, player_canonical, reference_date, days=14):
    try:
        cutoff_date = pd.to_datetime(reference_date) - pd.Timedelta(days=days)

        player_matches = self.historical_data[
            ((self.historical_data['winner_canonical'] == player_canonical) |
             (self.historical_data['loser_canonical'] == player_canonical))
        ].copy()

        if len(player_matches) == 0:
            return player_matches

        # Force string conversion then datetime to avoid mixed types
        player_matches['Date'] = pd.to_datetime(player_matches['Date'].astype(str), errors='coerce')
        player_matches = player_matches.dropna(subset=['Date'])
        player_matches = player_matches[player_matches['Date'] >= cutoff_date]

        return player_matches.sort_values('Date')
    except:
        # Return empty DataFrame on any error
        return pd.DataFrame()

def check_recent_retirements(self, player_canonical, reference_date):
    """Count recent retirements/walkovers - placeholder for actual retirement tracking"""
    # Implementation depends on your data structure for retirement flags
    return 0

In [99]:
## LAYER 3 ##
def simulate_match(self, player1_priors, player2_priors, best_of=3, tiebreak_sets=[1,2,3]):
    """Layer 3: Monte Carlo match simulation with Bayesian priors"""

    wins = 0
    simulations = self.simulation_count

    for _ in range(simulations):
        sets_won = [0, 0]  # [player1, player2]

        while max(sets_won) < (best_of + 1) // 2:
            set_winner = self.simulate_set(
                player1_priors,
                player2_priors,
                tiebreak=len([s for s in sets_won if s > 0]) + 1 in tiebreak_sets
            )
            sets_won[set_winner] += 1

        if sets_won[0] > sets_won[1]:
            wins += 1

    return wins / simulations

def simulate_set(self, p1_priors, p2_priors, tiebreak=True):
    """Simulate single set with service alternation"""
    games = [0, 0]
    server = 0  # 0 = player1 serves first

    while True:
        # Determine game winner based on server
        if server == 0:
            hold_prob = p1_priors['hold_prob']
            game_winner = 0 if np.random.random() < hold_prob else 1
        else:
            hold_prob = p2_priors['hold_prob']
            game_winner = 1 if np.random.random() < hold_prob else 0

        games[game_winner] += 1
        server = 1 - server  # Alternate serve

        # Check set completion
        if games[0] >= 6 and games[0] - games[1] >= 2:
            return 0
        elif games[1] >= 6 and games[1] - games[0] >= 2:
            return 1
        elif games[0] == 6 and games[1] == 6 and tiebreak:
            return self.simulate_tiebreak(p1_priors, p2_priors)

def simulate_tiebreak(self, p1_priors, p2_priors):
    """Simulate tiebreak with point-by-point serve alternation"""
    points = [0, 0]
    server = 0
    serve_count = 0

    while True:
        # Determine point winner
        if server == 0:
            hold_prob = p1_priors['hold_prob']
            point_winner = 0 if np.random.random() < hold_prob else 1
        else:
            hold_prob = p2_priors['hold_prob']
            point_winner = 1 if np.random.random() < hold_prob else 0

        points[point_winner] += 1
        serve_count += 1

        # Alternate server every 2 points (except first point)
        if serve_count == 1 or serve_count % 2 == 0:
            server = 1 - server

        # Check tiebreak completion
        if points[0] >= 7 and points[0] - points[1] >= 2:
            return 0
        elif points[1] >= 7 and points[1] - points[0] >= 2:
            return 1

def simulate_match(self, player1_priors, player2_priors, best_of=3, tiebreak_sets=[1,2,3]):
    wins = 0
    simulations = self.simulation_count

    for _ in range(simulations):
        sets_won = [0, 0]

        while max(sets_won) < (best_of + 1) // 2:
            set_winner = self.simulate_set(
                player1_priors,
                player2_priors,
                tiebreak=len([s for s in sets_won if s > 0]) + 1 in tiebreak_sets
            )
            sets_won[set_winner] += 1

        if sets_won[0] > sets_won[1]:
            wins += 1

    return wins / simulations

def simulate_set(self, p1_priors, p2_priors, tiebreak=True):
    games = [0, 0]
    server = 0

    while True:
        if server == 0:
            hold_prob = p1_priors['hold_prob']
            game_winner = 0 if np.random.random() < hold_prob else 1
        else:
            hold_prob = p2_priors['hold_prob']
            game_winner = 1 if np.random.random() < hold_prob else 0

        games[game_winner] += 1
        server = 1 - server

        if games[0] >= 6 and games[0] - games[1] >= 2:
            return 0
        elif games[1] >= 6 and games[1] - games[0] >= 2:
            return 1
        elif games[0] == 6 and games[1] == 6 and tiebreak:
            return self.simulate_tiebreak(p1_priors, p2_priors)

def simulate_tiebreak(self, p1_priors, p2_priors):
    points = [0, 0]
    server = 0
    serve_count = 0

    while True:
        if server == 0:
            hold_prob = p1_priors['hold_prob']

In [39]:
# Tomorrow's slate

import requests
from datetime import date, timedelta

API_KEY = "adfc70491c47895e5fffdc6428bbf36a561989d4bffcfa9ecfba8d91e947b4fb"
BASE = "https://api.api-tennis.com/tennis/"

def get_matches_for_date(target_date):
    params = {
        "method": "get_fixtures",
        "APIkey": API_KEY,
        "date_start": target_date,
        "date_stop": target_date
    }
    response = requests.get(BASE, params=params)
    if response.status_code != 200:
        raise RuntimeError(f"HTTP {response.status_code}")

    # Surface mapping
    TOURNAMENT_SURFACES = {
        'ATP Wimbledon': 'Grass',
        'WTA Wimbledon': 'Grass',
        'ATP French Open': 'Clay',
        'WTA French Open': 'Clay',
        'ATP US Open': 'Hard',
        'WTA US Open': 'Hard',
        'ATP Australian Open': 'Hard',
        'WTA Australian Open': 'Hard'
    }

    data = response.json()
    matches = []

    for event in data.get("result", []):
        matches.append({
            'event_key': event.get('event_key'),
            'player1_name': event['event_first_player'],
            'player2_name': event['event_second_player'],
            'tournament_name': event.get('tournament_name', 'Unknown'),
            'tournament_round': event.get('tournament_round', ''),
            'event_status': event.get('event_status', ''),
            'event_type_type': event.get('event_type_type', ''),
            'surface': TOURNAMENT_SURFACES.get(event.get('tournament_name', ''), 'Unknown'),
            'time': event.get('event_time', ''),
            'date': event.get('event_date', '')
        })

    return matches

def get_high_confidence_matches(target_date, min_confidence=0.2):
    matches = get_matches_for_date(target_date)

    results = []
    for match in matches:
        p1_canonical = convert_to_canonical(match['player1_name'])
        p2_canonical = convert_to_canonical(match['player2_name'])

        p1_priors = model.extract_refined_priors(p1_canonical, 'men', match['surface'], target_date)
        p2_priors = model.extract_refined_priors(p2_canonical, 'men', match['surface'], target_date)

        p1_win_prob = model.simulate_match(p1_priors, p2_priors)
        confidence = abs(p1_win_prob - 0.5)

        if confidence >= min_confidence:
            favorite = match['player1_name'] if p1_win_prob > 0.5 else match['player2_name']
            win_prob = max(p1_win_prob, 1 - p1_win_prob)

            results.append({
                'match': f"{match['player1_name']} vs {match['player2_name']}",
                'favorite': favorite,
                'probability': win_prob,
                'confidence': confidence
            })

    return sorted(results, key=lambda x: x['confidence'], reverse=True)

# Usage
today = date.today().isoformat()
tomorrow = (date.today() + timedelta(days=1)).isoformat()

todays_matches = get_matches_for_date(today)
tomorrows_matches = get_matches_for_date(tomorrow)

In [40]:
# Todays_matches or tomorrows_matches
todays_matches

[{'event_key': 12049946,
  'player1_name': 'M. Cilic',
  'player2_name': 'F. Cobolli',
  'tournament_name': 'ATP Wimbledon',
  'tournament_round': 'ATP Wimbledon - 1/8-finals',
  'event_status': '',
  'event_type_type': 'Atp Singles',
  'surface': 'Grass',
  'time': '12:00',
  'date': '2025-07-07'},
 {'event_key': 12049947,
  'player1_name': 'A. De Minaur',
  'player2_name': 'N. Djokovic',
  'tournament_name': 'ATP Wimbledon',
  'tournament_round': 'ATP Wimbledon - 1/8-finals',
  'event_status': '',
  'event_type_type': 'Atp Singles',
  'surface': 'Grass',
  'time': '14:30',
  'date': '2025-07-07'},
 {'event_key': 12049948,
  'player1_name': 'B. Shelton',
  'player2_name': 'L. Sonego',
  'tournament_name': 'ATP Wimbledon',
  'tournament_round': 'ATP Wimbledon - 1/8-finals',
  'event_status': '',
  'event_type_type': 'Atp Singles',
  'surface': 'Grass',
  'time': '15:30',
  'date': '2025-07-07'},
 {'event_key': 12049949,
  'player1_name': 'J. Sinner',
  'player2_name': 'G. Dimitrov',
  

In [41]:
# Get top 5 picks
def get_top_confidence_matches(target_date, top_n=5, min_confidence=0.05):
    matches = get_matches_for_date(target_date)

    results = []
    for match in matches:
        p1_canonical = convert_to_canonical(match['player1_name'])
        p2_canonical = convert_to_canonical(match['player2_name'])

        p1_priors = model.extract_refined_priors(p1_canonical, 'men', match['surface'], target_date)
        p2_priors = model.extract_refined_priors(p2_canonical, 'men', match['surface'], target_date)

        p1_win_prob = model.simulate_match(p1_priors, p2_priors)
        confidence = abs(p1_win_prob - 0.5)

        if confidence >= min_confidence:
            favorite = match['player1_name'] if p1_win_prob > 0.5 else match['player2_name']
            win_prob = max(p1_win_prob, 1 - p1_win_prob)

            results.append({
                'match': f"{match['player1_name']} vs {match['player2_name']}",
                'favorite': favorite,
                'probability': win_prob,
                'confidence': confidence
            })

    return sorted(results, key=lambda x: x['confidence'], reverse=True)[:top_n]

if __name__ == "__main__":
    target_date = date.today().isoformat()  # today's matches
    picks = get_top_confidence_matches(target_date, top_n=5, min_confidence=0.15)

    for i, pick in enumerate(picks, 1):
        print(f"{i}. {pick['match']}")
        print(f"   Favorite: {pick['favorite']}")
        print(f"   Win Prob: {pick['probability']:.2%}")
        print(f"   Confidence: {pick['confidence']:.5%}\n")

In [42]:
# See picks
from datetime import date

# get today’s top-5 at 5% confidence
picks = get_top_confidence_matches(date.today().isoformat(), top_n=5, min_confidence=0.05)

# print them
for i, pick in enumerate(picks, 1):
    print(f"{i}. {pick['match']}")
    print(f"   Favorite: {pick['favorite']}")
    print(f"   Win Prob: {pick['probability']:.2%}")
    print(f"   Confidence: {pick['confidence']:.1%}\n")

In [43]:
import pandas as pd

pd.DataFrame(picks)

""


In [44]:
# Split data chronologically
split_date = '2023-01-01'
train_data = historical_data[pd.to_datetime(historical_data['Date']) < split_date]
test_data = historical_data[pd.to_datetime(historical_data['Date']) >= split_date]

# Initialize model with training data
model.historical_data = train_data

# Run evaluation
accuracy = model.evaluate_predictions(test_data.head(100))
print(f"Enhanced model accuracy: {accuracy:.3f}")

# Compare with baseline
model_baseline = BayesianTennisModel()
model_baseline.historical_data = train_data
baseline_accuracy = model_baseline.evaluate_predictions(test_data.head(100))
print(f"Baseline accuracy: {baseline_accuracy:.3f}")
print(f"Improvement: {accuracy - baseline_accuracy:.3f}")

Enhanced model accuracy: 0.670
Baseline accuracy: 0.670
Improvement: 0.000


Tennis Abstract Scraper

In [108]:
import requests
from bs4 import BeautifulSoup, FeatureNotFound
import re
import pandas as pd
import html

class TennisAbstractScraper:
    def __init__(self):
        self.headers = {"User-Agent": "Mozilla/5.0"}

    # Stats Overview
    def scrape_stats_overview(self, url):
        resp = requests.get(url, headers=self.headers)
        try:
            soup = BeautifulSoup(resp.text, "lxml")
        except FeatureNotFound:
            soup = BeautifulSoup(resp.text, "html.parser")
        scripts = [tag.string for tag in soup.find_all("script") if tag.string]
        all_js = "\n".join(scripts)
        blocks = dict(re.findall(r"var\s+(\w+)\s*=\s*'([\s\S]*?)';", all_js, flags=re.S))
        labels = {span.get_text(strip=True): span["id"] for span in soup.select("span.rounds")}
        sections = {label: html.unescape(blocks[token]) for label, token in labels.items() if token in blocks}

        match_info = self._parse_match_url(url)
        stats_html = sections.get("Stats Overview", "")
        stats_data = self._extract_stats_overview_table(stats_html)

        # Return the raw parsed data for now; skip Jeff‑format conversion
        return self._convert_to_jeff_format(stats_data, match_info)

    # --- generic cell parser ------------------------------------------
    def _to_int(self, text):
        """
        Extract leading integer from a cell such as '51  (52%)' or '0'.
        Returns 0 when no digit is found.
        """
        m = re.search(r"\d+", text)
        return int(m.group(0)) if m else 0

    def _parse_match_url(self, url: str) -> dict:
        """
        Extract basic match metadata (date, tournament, player names) from a
        Tennis Abstract charting URL.

        Example URL
        -----------
        https://www.tennisabstract.com/charting/20250628-W-Eastbourne-F-Maya_Joint-Alexandra_Eala.html

        Returns
        -------
        dict
            {
                "Date": "20250628",
                "tournament": "Eastbourne",
                "player1": "Maya Joint",
                "player2": "Alexandra Eala"
            }
            An empty dict is returned when the pattern doesn’t match.
        """
        import re

        m = re.search(r"(\\d{8})-[MW]-(.+?)-[^-]+-([^-]+)_([^.]+)\\.html", url)
        if not m:
            return {}

        date_str, tour, p1, p2 = m.groups()
        return {
            "Date": date_str,
            "tournament": tour.replace("_", " "),
            "player1": p1.replace("_", " "),
            "player2": p2.replace("_", " ")
        }


    class TennisAbstractScraper:
        def scrape_serve_influence(self, url):
            resp  = requests.get(url, headers=self.headers)
            soup  = BeautifulSoup(resp.text, "lxml")

            js    = "\n".join(t.string for t in soup.find_all("script") if t.string)
            html_ = html.unescape(
                        dict(re.findall(r"var\s+(\w+)\s*=\s*'([\s\S]*?)';", js, re.S))
                        .get("serveloc", "")
                    )
            if not html_:
                return []

            tbl   = BeautifulSoup(html_, "html.parser").table
            heads = [h.get_text(strip=True).rstrip('%').strip()
                     for h in tbl.tr.find_all(['th', 'td'])]

            out = []
            for tr in tbl.find_all("tr")[1:]:
                cells  = [c.get_text(strip=True) for c in tr.find_all("td")]
                if not cells or not cells[0]:
                    continue
                rec = {"Player_canonical": self._normalize_player_name(cells[0])}
                for h, v in zip(heads[1:], cells[1:]):
                    key = self.MAP_SERVE_INFL.get(h)         # now matches
                    if key:
                        rec[key] = float(v.rstrip('%')) / 100
                out.append(rec)
            return out


        def scrape_serve_basics(self, url):
            """Return a list[dict] – one record per player × set (incl. Total)."""
            resp = requests.get(url, headers=self.headers)
            try:
                soup = BeautifulSoup(resp.text, "lxml")
            except FeatureNotFound:          # fall back if lxml not installed
                soup = BeautifulSoup(resp.text, "html.parser")

            # --- pull the raw HTML the page embeds in JS -------------------
            js   = "\n".join(t.string for t in soup.find_all("script") if t.string)
            blob = dict(re.findall(r"var\s+(\w+)\s*=\s*'([\s\S]*?)';", js, re.S))
            serve_html = html.unescape(blob.get("serve", ""))     # <- key fix
            if not serve_html:
                return []                                         # nothing found

            # --- parse the table ------------------------------------------
            tbl   = BeautifulSoup(serve_html, "html.parser").table
            rows  = [[c.get_text(strip=True) for c in tr.find_all(["th", "td"])]
                     for tr in tbl.find_all("tr")]
            hdr   = rows[0]
            data  = rows[1:]

            # --- reshape to Jeff format -----------------------------------
            match = self._parse_match_url(url)
            out   = []
            current_set = "Total"
            for r in data:
                if r[0].startswith("SET"):
                    current_set = r[0]
                    continue
                player = r[0]
                record = {
                    "match_id"        : f"{match['Date']}-{player.replace(' ', '_')}",
                    "Date"            : match["Date"],
                    "Tournament"      : match["tournament"],
                    "player"          : player,
                    "Player_canonical": self._normalize_player_name(player),
                    "set"             : current_set,
                    "serve_pts"       : int(r[1]),
                    "aces"            : int(r[2]),
                    "dfs"             : int(r[3]),
                    "first_in"        : int(r[4]),
                    "first_won"       : int(r[5]),
                    "second_won"      : int(r[6]),
                }
                out.append(record)
            return out

        def test_extraction_completeness(self, url):
            """Test all available sections and validate data structure"""
            sections = self.debug_available_sections(url)

            results = {}
            for section_name in sections.keys():
                try:
                    # Test each section extraction
                    extracted_data = self._test_section_extraction(url, section_name)
                    results[section_name] = len(extracted_data) > 0
                except Exception as e:
                    results[section_name] = f"Error: {e}"

            return results
        def _extract_stats_overview_table(self, html: str) -> list[dict]:
            return []

In [109]:
# Test both methods
scraper = TennisAbstractScraper()
url = "https://www.tennisabstract.com/charting/20250628-W-Eastbourne-F-Maya_Joint-Alexandra_Eala.html"

# Test Stats Overview
print("=== STATS OVERVIEW ===")
overview_data = scraper.scrape_stats_overview(url)
for record in overview_data:
    print(record)

print("\n=== SERVE BASICS ===")
serve_data = scraper.scrape_serve_basics(url)
for record in serve_data:
    print(record)

print("\n=== SERVE INFLUENCE ===")
serve_infl_data = scraper.scrape_serve_influence(url)
for record in serve_infl_data:
    print(record)

print("\n=== SERVE DIRECTION ===")
for rec in scraper.scrape_serve_direction(url):
    print(rec)

print("\n=== RETURN BREAKDOWN ===")
for rec in scraper.scrape_return_breakdown(url):
    print(rec)

print("\n=== NET POINTS ===")
for rec in scraper.scrape_net_points(url):
    print(rec)


=== STATS OVERVIEW ===


AttributeError: 'TennisAbstractScraper' object has no attribute '_extract_stats_overview_table'